---
title: Centum Getting Start
subtitle: Water Accounting usnig Centum
license: CC-BY-4.0
github: https://github.com/executablebooks/mystjs
subject: Tutorial
authors:
  - name: Benjamin Mary
    email: benjamin.mary@ica.csic.es
    corresponding: true
    orcid: 
    affiliations:
      - ICA-CSIC
date: 2025/01/10
---

## Welcome to using Centum 

:::{important} Installing Centum
```bash
pip install centum
```
:::

In [4]:
!pip install centum==0.1.4
#!python ../setup.py install

  Attempting uninstall: centum
    Found existing installation: centum 0.3
    Uninstalling centum-0.3:
      Successfully uninstalled centum-0.3


In [6]:
from centum import plotting as pltC
import numpy as np
import xarray as xr

from centum.irrigation_district import IrrigationDistrict 


In [5]:

from centum.delineation import ETAnalysis
import matplotlib.pyplot as plt

from centum.irrigation_district import IrrigationDistrict 
from centum import irrigation_district as irr_geo_tools 


ImportError: cannot import name 'ETAnalysis' from 'centum.delineation' (/home/z0272571a@CAMPUS.CSIC.ES/miniconda3/envs/Bousval/lib/python3.9/site-packages/centum/delineation.py)

## Open datasets

In [ ]:
ds_analysis_EO = xr.open_dataset(f'../data/synthetic/ds_analysis_EO_0.netcdf')
ds_analysis_baseline = xr.open_dataset(f'../data/synthetic/ds_analysis_baseline_0.netcdf')

## Show Earth Observation time serie to analyse

In [ ]:
_, index = np.unique(ds_analysis_EO['time'], return_index=True)
ds_analysis_EO = ds_analysis_EO.isel(time=index)
ds_analysis_baseline = ds_analysis_baseline.isel(time=index)

ds_analysis_EO['ACT. ETRA'].plot.imshow(x="x", y="y", 
                                        col="time", 
                                        col_wrap=4,
                                        )
ds_analysis_baseline['ACT. ETRA'].plot.imshow(x="x", y="y", 
                                        col="time", 
                                        col_wrap=4,
                                        )

## Irrigation delineation

In [ ]:
scenario_analysis_usingET = ETAnalysis()
decision_ds, event_type = scenario_analysis_usingET.irrigation_delineation(ds_analysis_EO)

#%% Plot timeline 
ncols = 4
time_steps = event_type.time.size
nrows = int(np.ceil(time_steps / ncols))  # Number of rows needed
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, 
                         figsize=(15, nrows * 3)
                         )
pltC.plot_irrigation_schedule(event_type,time_steps,fig,axes)

## Prepare geographic analysis of irrigation districts

In [ ]:
GeoAnalysis = IrrigationDistrict("../data/synthetic/area_SC0.shp")
gdf_irr = GeoAnalysis.load_shapefile()
resolution = event_type.rio.resolution()[0]
bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)
gdf_irr['index'] = gdf_irr.index.values
rioxr_irrigation = GeoAnalysis.convert_to_rioxarray(gdf=gdf_irr, 
                                                    variable="index", 
                                                    resolution=resolution, 
                                                    bounds=bounds
                                                    )
rioxr_irrigation = rioxr_irrigation.rio.write_crs("EPSG:4326")
event_type = event_type.rio.write_crs("EPSG:4326")
rioxr_irrigation= rioxr_irrigation.rio.reproject_match(event_type)


## Show result for a given district

In [ ]:
mask_IN = irr_geo_tools.get_mask_IN_patch_i(rioxr_irrigation,
                                    patchid=1
                                    )
mask_IN = rioxr_irrigation==1
mask_OUT = irr_geo_tools.get_mask_OUT(rioxr_irrigation,
                                      )

event_type_node_IN = event_type.where(mask_IN, drop=True).mean(['x','y'])
event_type_node_OUT = event_type.where(mask_OUT, drop=True).mean(['x','y'])